In [1]:
using Plots
using GaussianProcesses
using Optim
using NPZ
using Distributions
using StatsPlots
using BenchmarkTools

In [2]:
function get_optimized_gp(X, y)
    mZero = MeanZero()
    kern = SEArd([0.0,0.0],0.0)
    
    # create a gp and fit to data
    gp = GP(X,y,mZero,kern) 
    
    optimize!(gp)
    
    lat, long = sqrt.(1.0./gp.kernel.iℓ2)
    sigma = sqrt(gp.kernel.σ2)
    
    return sigma, lat, long
end

get_optimized_gp (generic function with 1 method)

In [3]:
function get_optimized_gp_iso(X, y)
    mZero = MeanZero()
    kern = SE(0.0,0.0)
    
    # create a gp and fit to data
    gp = GP(X,y,mZero,kern) 
    
    optimize!(gp)
    
    lat = sqrt(gp.kernel.ℓ2)
    sigma = sqrt(gp.kernel.σ2)
    
    return sigma, lat
end

get_optimized_gp_iso (generic function with 1 method)

In [4]:
function read_sampled_data(data_path, ktype)
    # read in the sampled data
    vars = npzread(data_path)
    data = vars["data"]
    y = vars["y"]
    # fit GP
    if (ktype == "ard")
        return @btime get_optimized_gp(transpose($data), $y)
    else
        return @btime get_optimized_gp_iso(transpose($data), $y)
    end

end

read_sampled_data (generic function with 1 method)

In [5]:
# test GP
#Training data
d, n = 2, 400;         #Dimension and number of observations
x = 2π * rand(d, n);                               #Predictors
y = vec(sin.(x[1,:]).*sin.(x[2,:])) + 0.05*rand(n);  #Responses

In [6]:
println(size(x),size(y))

(2, 400)(400,)


In [7]:
mZero = MeanZero()
kern = SEArd([0.0,0.0],0.0)
gp = GP(x,y,mZero,kern) 

GP Exact object:
  Dim = 2
  Number of observations = 400
  Mean function:
    Type: MeanZero, Params: Float64[]
  Kernel:
    Type: SEArd{Float64}, Params: [-0.0, -0.0, 0.0]
  Input observations = 
[1.061614749814572 1.154540263557316 … 4.634970287348769 0.3635695671411009; 3.927761391768369 4.446007130786972 … 4.458038427632716 4.977679683109724]
  Output observations = [-0.5808213595489996, -0.8583360823136968, -0.21271305479494168, 0.6532027534660715, 0.19748079767319912, -0.08089312997888469, 0.009808111203733549, -0.9085345618791008, 0.25774566253967846, -0.12460284173388178  …  -0.2849036362092231, -0.6628583491995882, 0.5327091871317519, 0.04234598942626888, 0.11361838123040202, -0.6856628711932914, 0.1908015068912695, 0.4342412375104539, 1.0111788019216608, -0.33000412782984495]
  Variance of observation noise = 0.01831563888873418
  Marginal Log-Likelihood = 292.722

In [8]:
optimize!(gp)

LinearAlgebra.PosDefException(3)
LinearAlgebra.PosDefException(9)


 * Status: success

 * Candidate solution
    Final objective value:     -9.863208e+02

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 3.25e-10 ≰ 0.0e+00
    |x - x'|/|x'|          = 7.63e-11 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.35e-10 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.39e-13 ≰ 0.0e+00
    |g(x)|                 = 4.76e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    15
    f(x) calls:    43
    ∇f(x) calls:   43


In [9]:
gp.kernel

Type: SEArd{Float64}, Params: [0.8106875284265732, 0.7882993441445321, 0.6293505800163909]

In [10]:
gp.kernel.iℓ2

2-element Array{Float64,1}:
 0.19762676412707125
 0.2066768765254109

In [11]:
gp.kernel.σ2

3.520845501416324

In [12]:
print(sqrt.(1.0./gp.kernel.iℓ2),exp.([0.8125298099124523, 0.8126645194321461]))

[2.249454018727224, 2.199652391781168][2.2536019658892137, 2.2539055679762066]

In [13]:
pwd()

"/Users/dbanesh/Documents/insitInference/dataML"

In [14]:
data_dir = pwd() * "/Data2/output_data2"

"/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2"

In [16]:
sigma_vec = Any[]
lat_vec = Any[]
long_vec = Any[]
file_vec = Any[]
count = 0
for f in filter(x -> endswith(x, "npz"), readdir(data_dir))
    println(data_dir *"/" * f)
    count += 1
    try
        sigma, lat, long = read_sampled_data(data_dir *"/" * f, "ard")
        push!(sigma_vec, sigma)
        push!(lat_vec, lat)
        push!(long_vec, long)
        push!(file_vec, splitext(f)[1])
    catch
        continue
    
    end
end

/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample0.npz
  6.053 s (3159 allocations: 465.50 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample1.npz
  7.776 s (4000 allocations: 593.93 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample10.npz
  9.946 s (4005 allocations: 585.90 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample100.npz
  8.087 s (3251 allocations: 473.53 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample101.npz
  11.825 s (3836 allocations: 561.82 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample102.npz
  11.144 s (4208 allocations: 618.01 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample103.npz
  10.103 s (3704 allocations: 545.77 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample104.npz
  7.089 s (3720 allocations: 545.77 MiB)
/Users/dbanesh/Documents/insitInfe

  12.736 s (4825 allocations: 706.30 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample155.npz
  7.462 s (3093 allocations: 457.48 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample156.npz
  9.100 s (3694 allocations: 545.77 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample157.npz
  9.990 s (3594 allocations: 529.72 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample158.npz
  9.634 s (3776 allocations: 553.80 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample159.npz
  9.931 s (3813 allocations: 561.82 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample16.npz
  7.928 s (3470 allocations: 505.64 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample160.npz
  9.044 s (3902 allocations: 569.85 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample161.npz
  8.784 s (3683 allocations: 537

LinearAlgebra.PosDefException(1)
LinearAlgebra.PosDefException(1)
LinearAlgebra.PosDefException(12)
LinearAlgebra.PosDefException(264)
LinearAlgebra.PosDefException(10)
LinearAlgebra.PosDefException(1)
LinearAlgebra.PosDefException(1)
LinearAlgebra.PosDefException(12)
LinearAlgebra.PosDefException(264)
LinearAlgebra.PosDefException(10)
  10.588 s (4669 allocations: 569.76 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample196.npz
LinearAlgebra.PosDefException(195)
LinearAlgebra.PosDefException(195)
LinearAlgebra.PosDefException(195)
LinearAlgebra.PosDefException(195)
  11.865 s (4577 allocations: 634.05 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample197.npz
LinearAlgebra.PosDefException(66)
LinearAlgebra.PosDefException(66)
LinearAlgebra.PosDefException(66)
LinearAlgebra.PosDefException(66)
  11.065 s (4248 allocations: 593.92 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample198.npz
LinearAlgebra.PosDef

  9.399 s (4029 allocations: 593.93 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample240.npz
  10.239 s (3770 allocations: 553.80 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample241.npz
  11.888 s (4129 allocations: 609.98 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample242.npz
LinearAlgebra.PosDefException(22)
LinearAlgebra.PosDefException(190)
LinearAlgebra.PosDefException(6)
LinearAlgebra.PosDefException(22)
LinearAlgebra.PosDefException(190)
LinearAlgebra.PosDefException(6)
LinearAlgebra.PosDefException(22)
LinearAlgebra.PosDefException(190)
LinearAlgebra.PosDefException(6)
LinearAlgebra.PosDefException(22)
LinearAlgebra.PosDefException(190)
LinearAlgebra.PosDefException(6)
  9.439 s (4285 allocations: 553.75 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample243.npz
LinearAlgebra.PosDefException(10)
LinearAlgebra.PosDefException(10)
LinearAlgebra.PosDefException(10)
Line

  8.093 s (3879 allocations: 569.85 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample267.npz
LinearAlgebra.PosDefException(4)
LinearAlgebra.PosDefException(4)
LinearAlgebra.PosDefException(4)
LinearAlgebra.PosDefException(4)
  19.857 s (6595 allocations: 939.06 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample268.npz
  8.005 s (3923 allocations: 577.88 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample269.npz
  8.728 s (3908 allocations: 569.85 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample27.npz
  9.255 s (3810 allocations: 561.82 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample270.npz
  14.103 s (5271 allocations: 778.54 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample271.npz
  12.559 s (4053 allocations: 601.95 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample272.npz
  11.059 s (3786 al

  8.553 s (3544 allocations: 521.69 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample322.npz
LinearAlgebra.PosDefException(65)
LinearAlgebra.PosDefException(385)
LinearAlgebra.PosDefException(961)
LinearAlgebra.PosDefException(65)
LinearAlgebra.PosDefException(385)
LinearAlgebra.PosDefException(961)
LinearAlgebra.PosDefException(65)
LinearAlgebra.PosDefException(385)
LinearAlgebra.PosDefException(961)
LinearAlgebra.PosDefException(65)
LinearAlgebra.PosDefException(385)
LinearAlgebra.PosDefException(961)
  6.371 s (3561 allocations: 449.40 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample323.npz
  10.555 s (4108 allocations: 601.96 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample324.npz
LinearAlgebra.PosDefException(3)
LinearAlgebra.PosDefException(3)
LinearAlgebra.PosDefException(3)
LinearAlgebra.PosDefException(3)
  14.774 s (5324 allocations: 754.45 MiB)
/Users/dbanesh/Documents/insitInference/dataML

LinearAlgebra.PosDefException(9)
LinearAlgebra.PosDefException(14)
  5.660 s (3818 allocations: 513.63 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample364.npz
LinearAlgebra.PosDefException(15)
LinearAlgebra.PosDefException(17)
LinearAlgebra.PosDefException(15)
LinearAlgebra.PosDefException(17)
LinearAlgebra.PosDefException(15)
LinearAlgebra.PosDefException(17)
LinearAlgebra.PosDefException(15)
LinearAlgebra.PosDefException(17)
  11.203 s (5153 allocations: 714.29 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample365.npz
  6.476 s (3789 allocations: 553.80 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample366.npz
  9.199 s (4595 allocations: 682.22 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample367.npz
LinearAlgebra.PosDefException(9)
LinearAlgebra.PosDefException(9)
LinearAlgebra.PosDefException(9)
LinearAlgebra.PosDefException(9)
LinearAlgebra.PosDefException(9)
LinearAlgeb

  6.842 s (3647 allocations: 537.74 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample4.npz
  6.219 s (3836 allocations: 561.82 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample40.npz
  6.378 s (3673 allocations: 537.74 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample41.npz
  5.863 s (3441 allocations: 505.64 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample42.npz
  6.396 s (3760 allocations: 553.80 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample43.npz
LinearAlgebra.PosDefException(193)
LinearAlgebra.PosDefException(193)
LinearAlgebra.PosDefException(193)
LinearAlgebra.PosDefException(193)
  6.276 s (3966 allocations: 553.78 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample44.npz
LinearAlgebra.PosDefException(3)
LinearAlgebra.PosDefException(3)
LinearAlgebra.PosDefException(3)
LinearAlgebra.PosDefException(3)
  5.510 s

  5.772 s (3533 allocations: 513.67 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample80.npz
  8.024 s (4121 allocations: 601.96 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample81.npz
  6.971 s (3636 allocations: 529.72 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample82.npz
  6.016 s (3670 allocations: 537.74 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample83.npz
  6.235 s (3902 allocations: 569.85 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample84.npz
LinearAlgebra.PosDefException(3)
LinearAlgebra.PosDefException(3)
LinearAlgebra.PosDefException(3)
LinearAlgebra.PosDefException(3)
  7.352 s (4261 allocations: 593.92 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample85.npz
  7.352 s (4356 allocations: 634.07 MiB)
/Users/dbanesh/Documents/insitInference/dataML/Data2/output_data2/sample86.npz
  5.990 s (3657 allocations:

In [ ]:
using Base.MathConstants
histogram(sigma_vec)

In [ ]:
histogram(lat_vec)

In [ ]:
histogram(long_vec)

In [ ]:
density(sigma_vec)

In [ ]:
density(lat_vec)

In [ ]:
density(long_vec)

In [ ]:
sigma_vec = convert(Array{Float64,1}, sigma_vec)
lat_vec = convert(Array{Float64,1}, lat_vec)
long_vec = convert(Array{Float64,1}, long_vec)

file_numbers = []
for str in file_vec
    number_as_string=split(str,"e")[end]
    push!(file_numbers, parse(Int, number_as_string))
end
file_numbers = convert(Array{Int64,1}, file_numbers)

npzwrite("param_ard_data2.npz", Dict("fileNum" => file_numbers, "sigma" => sigma_vec, "lat" => lat_vec, "long" => long_vec))